<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/OpenMM/OpenMM_Scripting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#new stuff

In [15]:
!pip install -q condacolab
import condacolab
condacolab.install()  # restarts runtime; rerun from here after restart

!mamba install -c conda-forge ambertools -y

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...

Looking for: ['ambertools']

[+] 0.0s
conda-forge/linux-64  ⣾  [+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64  ⣾  
conda-forge/noarch     1%[+] 0.3s
conda-forge/linux-64   1%
conda-forge/noarch     1%[+] 0.4s
conda-forge/linux-64   3%
conda-forge/noarch     6%[+] 0.5s
conda-forge/linux-64   8%
conda-forge/noarch    27%[+] 0.6s
conda-forge/linux-64  16%
conda-forge/noarch    46%[+] 0.7s
conda-forge/linux-64  19%
conda-forge/noarch    55%[+] 0.8s
conda-forge/linux-64  22%
conda-forge/noarch    59%[+] 0.9s
conda-forge/linux-64  26%
conda-forge/noarch    66%[+] 1.0s
conda-forge/linux-64  28%
conda-forge/noarch    72%[+] 1.1s
conda-forge/linux-64  32%
conda-forge/noarch    81%[+] 1.2s
conda-forge/linux

In [1]:
!git clone https://github.com/Iourarum/GOPY.git
%cd GOPY
!python GOPY.py generate_PG 10 10 graphene.pdb
!python GOPY.py generate_GO graphene.pdb 2 4 8 functionalized.pdb

Cloning into 'GOPY'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 197 (delta 39), reused 18 (delta 18), pack-reused 150 (from 1)
Receiving objects: 100% (197/197), 9.50 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (99/99), done.
/content/GOPY
done.
GOPY.py
generate_GO
graphene.pdb
2
4
8
functionalized.pdb
Left to add:  cooh:  2 epoxy:  4 hydroxyl:  8
Left to add:  cooh:  2 epoxy:  3 hydroxyl:  8
Left to add:  cooh:  1 epoxy:  3 hydroxyl:  8
Left to add:  cooh:  1 epoxy:  2 hydroxyl:  8
Left to add:  cooh:  1 epoxy:  2 hydroxyl:  7
Left to add:  cooh:  0 epoxy:  2 hydroxyl:  7
Left to add:  cooh:  0 epoxy:  1 hydroxyl:  7
Left to add:  cooh:  0 epoxy:  0 hydroxyl:  7
Left to add:  cooh:  0 epoxy:  0 hydroxyl:  6
Left to add:  cooh:  0 epoxy:  0 hydroxyl:  5
Left to add:  cooh:  0 epoxy:  0 hydroxyl:  4
Left to add:  cooh:  0 epoxy:  0 hydroxyl:  3
Left to add:  cooh:  0 epoxy:  0

In [2]:
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/GGG.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/C1A.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/E1A.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/H1A.lib
!mkdir -p FRCMOD
!wget -P FRCMOD https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/FRCMOD/GO.frcmod

--2025-01-21 05:08:47--  https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/GGG.lib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1063 (1.0K) [text/plain]
Saving to: ‘GGG.lib’

GGG.lib             100%[===================>]   1.04K  --.-KB/s    in 0s      

2025-01-21 05:08:47 (35.7 MB/s) - ‘GGG.lib’ saved [1063/1063]

--2025-01-21 05:08:47--  https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/C1A.lib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1627 (1.6K) [text/plain]
Saving to: ‘C1A.lib’

C1

In [18]:
%%bash
cd /content/GOPY

cat > leap.in <<EOF
source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadoff GGG.lib
loadoff C1A.lib
loadoff E1A.lib
loadoff H1A.lib
loadamberparams FRCMOD/GO.frcmod
mol = loadpdb functionalized.pdb
bondbydistance mol
solvateBox mol TIP3PBOX 10.0
addIons2 mol Na+ 40
addIons2 mol Cl- 40
saveamberparm mol mol_solv.prmtop mol_solv.inpcrd
quit
EOF
tleap -f leap.in



-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./leap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cmd/

In [8]:
!pip install openmm

  Using cached OpenMM-8.2.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (844 bytes)
Using cached OpenMM-8.2.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (12.3 MB)


In [19]:
from openmm.app import (
    AmberPrmtopFile,
    AmberInpcrdFile,
    Simulation,
    PME,
    PDBFile,
    HBonds
)
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import *

# Load solvated Amber parameters
prmtop = AmberPrmtopFile("/content/GOPY/mol_solv.prmtop")
inpcrd = AmberInpcrdFile("/content/GOPY/mol_solv.inpcrd")

# Extract topology and coordinates
topology = prmtop.topology
positions = inpcrd.positions

# Create the system (no need for ForceField('amber14-all.xml') now)
system = prmtop.createSystem(
    nonbondedMethod=PME,
    nonbondedCutoff=1.0*nanometer,
    constraints=HBonds
)

# Define integrator and select platform
integrator = LangevinMiddleIntegrator(
    300*kelvin,       # Temperature
    1/picosecond,     # Friction
    0.004*picoseconds # Timestep
)
platform = Platform.getPlatformByName("CPU")

# Initialize simulation
simulation = Simulation(topology, system, integrator, platform)
simulation.context.setPositions(positions)

# Minimize energy
simulation.minimizeEnergy()

# Run a short test
simulation.step(1000)

# Write out final structure
state = simulation.context.getState(getPositions=True)
with open("functionalized_graphene_with_water.pdb", "w") as f:
    PDBFile.writeFile(simulation.topology, state.getPositions(), f)

print("Simulation complete. Structure minimized with water and ions added.")

Simulation complete. Structure minimized with water and ions added.
